In [ ]:
! pip install opencv-python == 4.5.5.64

In [ ]:
import sys
from google.colab import drive

drive.mount('/content/drive')
sys.path.append(r"/content/drive/MyDrive/")

In [ ]:
# import the necessary packages
from loader_util.preprocessing import ImageToArrayPreprocessor
from loader_util.preprocessing import SimplePreprocessor
from loader_util.preprocessing import PatchPreprocessor
from loader_util.preprocessing import MeanPreprocessor
from loader_util.callbacks import TrainingMonitor
from loader_util.io import HDF5DatasetGenerator
from loader_util.nn.conv import AlexNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import json
import os

In [ ]:
# script constants
mean_json_path = r""
train_dataset_path = r""
valid_dataset_path = r""
batch_size = 128
num_epochs = 100
output_path = r""
model_output_path = os.path.sep.join([output_path, "saved_model.h5"])

In [ ]:
# construct the data augmentor
aug = ImageDataGenerator(rotation_range=20,
                         zoom_range=0.15,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         shear_range=0.15,
                         horizontal_flip=True,
                         fill_mode="nearest")

In [ ]:
# load the RGB means for the training set
means = json.loads(open(mean_json_path).read())

# initialize the image preprocessors
sp = SimplePreprocessor(227, 227)
pp = PatchPreprocessor(227, 227)
mp = MeanPreprocessor(means["R"], means["G"], means["B"])
iap = ImageToArrayPreprocessor()

In [ ]:
# init the train and valis dataset generators
train_gen = HDF5DatasetGenerator(dbpath=train_dataset_path,
                                 batch_size=batch_size,
                                 aug=aug,
                                 preprocessors=[pp, mp, iap])

val_gen = HDF5DatasetGenerator(dbpath=valid_dataset_path,
                               batch_size=batch_size,
                               preprocessors=[sp, mp, iap])

In [ ]:
# initialize the optimizer
print("[INFO] compiling model...")
opt = Adam(lr=1e-3)
model = AlexNet.build(width=227, height=227, depth=3, classes=2, reg=0.0002)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# construct the set of callbacks
path = os.path.sep.join([output_path, "{}.png".format(os.getpid())])
callbacks = [TrainingMonitor(path)]

In [ ]:
# train the network
H = model.fit_generator(train_gen.generator(),
                        steps_per_epoch=train_gen.num_images // batch_size,
                        validation_data=val_gen.generator(),
                        validation_steps=val_gen.numImages // batch_size,
                        epochs=num_epochs,
                        max_queue_size=10,
                        callbacks=callbacks,
                        verbose=1)

In [ ]:
# save the model to file
print("[INFO] serializing model...")
model.save(model_output_path, overwrite=True)
# close the HDF5 datasets
train_gen.close()
val_gen.close()